In [162]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [163]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [164]:

# %timeit sirve para evaluar el tiempo de ejecucion
train_entrenamiento = pd.read_csv('train_paso_3.csv')
test_entrenamiento = pd.read_csv('test_paso_3.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')


In [165]:
len(train_entrenamiento)
train_entrenamiento = train_entrenamiento.drop('person', 1)

In [166]:
test_entrenamiento = test_entrenamiento.drop('person', 1)
train_entrenamiento.head()

,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,...,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone,time,diftime
0,0,viewed product,23,checkout,1,6.0,3.0,1.0,1.0,15.0,...,2.0,0.0,0.0,0.0,11.0,1.0,0.0,11.0,772618.0,0.333333
1,0,ad campaign hit,1,ad campaign hit,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0,viewed product,31,checkout,1,9.0,14.0,1.0,0.0,9.0,...,0.0,0.0,0.0,0.0,14.0,0.0,0.0,18.0,9335392.0,9.457143
3,0,viewed product,24,search engine hit,1,0.0,5.0,0.0,0.0,3.0,...,0.0,6.0,5.0,0.0,10.0,3.0,0.0,0.0,742549.0,0.666667
4,0,viewed product,9,visited site,1,5.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,1409.0,0.333333


In [167]:
#hash
#Transformo texto a numeros
train_entrenamiento['most_frequent_event'] = train_entrenamiento.apply(lambda row:  hash(row['most_frequent_event'])\
                        if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
train_entrenamiento['less_frequent_event'] = train_entrenamiento.apply(lambda row:  hash(row['less_frequent_event'])\
                        if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [168]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

train_entrenamiento=normalize(train_entrenamiento)
train_entrenamiento.head()

,label,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,...,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone,time,diftime
0,0.0,-1.038073,0.009346,2.181286,0.0,0.011472,0.001586,0.015152,0.033333,0.031915,...,0.013158,0.000000,0.00000,0.0,0.010526,0.004926,0.0,0.005329,0.059450,0.001389
1,0.0,1.000000,0.000000,1.000000,0.0,0.001912,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,-1.038073,0.012744,2.181286,0.0,0.017208,0.007403,0.015152,0.000000,0.019149,...,0.000000,0.000000,0.00000,0.0,0.013397,0.000000,0.0,0.008721,0.718322,0.039405
3,0.0,-1.038073,0.009771,2.957752,0.0,0.000000,0.002644,0.000000,0.000000,0.006383,...,0.000000,0.085714,0.00615,0.0,0.009569,0.014778,0.0,0.000000,0.057136,0.002778
4,0.0,-1.038073,0.003398,-2.336989,0.0,0.009560,0.000000,0.030303,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.0,0.010526,0.000000,0.0,0.000000,0.000108,0.001389


In [169]:
string_columns=train_entrenamiento.columns
string_columns

Index(['label', 'most_frequent_event', 'count_most_frequent_event',
       'less_frequent_event', 'count_less_frequent_event', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing', 'lead',
       'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'mean_per_event', '128GB', '16GB',
       '256GB', '32GB', '4GB', '512MB', '64GB', '8GB', 'Bom',
       'Bom - Sem Touch ID', 'Excelente', 'Muito Bom', 'Novo',
       'desconocido_modelo', 'Asus', 'LG', 'Lenovo', 'Motorola', 'Quantum',
       'Samsung', 'Sony', 'iPad', 'iPhone', 'time', 'diftime'],
      dtype='object')

In [170]:
# #hash
# #Transformo texto a numeros
# train_entrenamiento['most_frequent_event'] = train_entrenamiento.apply(lambda row:  hash(row['most_frequent_event'])\
#                         if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
# train_entrenamiento['less_frequent_event'] = train_entrenamiento.apply(lambda row:  hash(row['less_frequent_event'])\
#                         if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [171]:
train = train_entrenamiento.drop('label', 1)
train.head()

,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,...,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone,time,diftime
0,-1.038073,0.009346,2.181286,0.0,0.011472,0.001586,0.015152,0.033333,0.031915,0.0,...,0.013158,0.000000,0.00000,0.0,0.010526,0.004926,0.0,0.005329,0.059450,0.001389
1,1.000000,0.000000,1.000000,0.0,0.001912,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,-1.038073,0.012744,2.181286,0.0,0.017208,0.007403,0.015152,0.000000,0.019149,0.0,...,0.000000,0.000000,0.00000,0.0,0.013397,0.000000,0.0,0.008721,0.718322,0.039405
3,-1.038073,0.009771,2.957752,0.0,0.000000,0.002644,0.000000,0.000000,0.006383,0.0,...,0.000000,0.085714,0.00615,0.0,0.009569,0.014778,0.0,0.000000,0.057136,0.002778
4,-1.038073,0.003398,-2.336989,0.0,0.009560,0.000000,0.030303,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00000,0.0,0.010526,0.000000,0.0,0.000000,0.000108,0.001389


In [172]:
trainlabel = train_entrenamiento['label']
trainlabel.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

In [173]:
# """"Devuelve un array de tuplas, donde el primer elemento de cada tupla es el score para esa combinacion de hiper-parametros,
# se usa cross-validation"""   

# layer_options = [1,5]

# scores_actuales = []

# for layer in layer_options :
# #     print "Ahora probando con: " + str(layer) + " capas de la red"
#     #cargo datos al modelo
            
#     model = MLPRegressor(activation='relu',solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(layer,),batch_size='auto', random_state=1)

#     scores = cross_val_score(model, train, trainlabel, cv=40,  scoring='neg_mean_squared_error', n_jobs = -1) 
#     scores_actuales.append((scores.mean(),[layer]))
# #return scores_actuales

In [174]:
# scores_actuales

In [175]:
# best_score = -float("inf") #Empiezo con un error infinito
# best_hiper_parameters = []

# for s in scores_actuales:
#     if s[0] > best_score:
#         best_score = s[0]
#         best_hiper_parameters = s[1]

In [176]:
# best_hiper_parameters[0]

In [202]:
# layer = best_hiper_parameters[0]
#cargo datos al modelo
clf = MLPRegressor(activation='relu',solver='lbfgs', alpha=1e-5,hidden_layer_sizes=1,batch_size='auto', random_state=1)
clf.fit(train, trainlabel)    
#return clf

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=20, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=7, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [203]:

parameters_for_testing = {
#    'colsample_bytree':[0.4,0.6,0.8],
#    'gamma':[0,0.03,0.1],
#    'min_child_weight':[1.5,6,10],
#    'learning_rate':[0.1,0.07,0.01],
#    'max_depth':[3,5,7],
#    'n_estimators':[10,100,1000],
#    'reg_alpha':[1e-5, 1e-2,  0.75],
#    'reg_lambda':[1e-5, 1e-2, 0.45],
#    'subsample':[0.6,0.95]
    'hidden_layer_sizes':[1,2,3,4,5,10,15,20,50],
    'random_state':[1,2,5,8,15],
}

In [204]:
# gsearch1 = GridSearchCV(estimator = clf, param_grid = parameters_for_testing,\
#                        n_jobs=6,iid=False, verbose=10,scoring='roc_auc')
# gsearch1.fit(train_entrenamiento,train_label)
# print (gsearch1.grid_scores_)
# print('best params')
# print (gsearch1.best_params_)
# print('best score')
# print (gsearch1.best_score_)

In [205]:
test_entrenamiento.head()

,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,mean_per_event,ad campaign hit,brand listing,checkout,conversion,generic listing,...,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone,time,diftime
0,-1.038073,0.001570,2.181286,0.000000,0.001016,0.0000,0.000000,0.005076,0.0,0.003745,...,0.000000,0.000000,0.000000,0.000000,0.004955,0.000000,0.00000,0.000000,0.045435,0.000000
1,-1.038073,0.210884,-3.235016,0.000000,0.141990,0.0725,0.002972,0.035533,0.0,0.078652,...,0.000000,0.000000,0.000000,0.000000,0.001982,0.000000,0.03876,0.257798,0.376711,0.005839
2,-1.038073,0.006279,1.000000,0.000000,0.004572,0.0025,0.003715,0.005076,0.0,0.003745,...,0.000000,0.000000,0.008089,0.000000,0.000000,0.000000,0.00000,0.003819,0.000204,0.002972
3,-1.038073,0.386185,2.181286,0.000000,0.186724,0.0375,0.005201,0.005076,0.0,0.074906,...,0.304527,0.051948,0.522750,0.130435,0.033697,0.162963,0.00000,0.005092,0.391638,0.007634
4,-1.038073,0.092098,2.181286,0.030303,0.063248,0.0350,0.005944,0.010152,0.0,0.052434,...,0.000000,0.000000,0.000000,0.000000,0.002973,0.000000,0.00000,0.112031,0.044474,0.004249


In [206]:
#hash
#Transformo texto a numeros
test_entrenamiento['most_frequent_event'] = test_entrenamiento.apply(lambda row:  hash(row['most_frequent_event'])\
                        if not(pd.isnull(row['most_frequent_event'])) else np.nan, axis = 1)
test_entrenamiento['less_frequent_event'] = test_entrenamiento.apply(lambda row:  hash(row['less_frequent_event'])\
                        if not(pd.isnull(row['less_frequent_event'])) else np.nan, axis = 1)

In [207]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

test_entrenamiento=normalize(test_entrenamiento)
test_entrenamiento.head()

,most_frequent_event,count_most_frequent_event,less_frequent_event,count_less_frequent_event,mean_per_event,ad campaign hit,brand listing,checkout,conversion,generic listing,...,LG,Lenovo,Motorola,Quantum,Samsung,Sony,iPad,iPhone,time,diftime
0,0.390027,0.001570,0.524391,0.000000,0.001016,0.0000,0.000000,0.005076,0.0,0.003745,...,0.000000,0.000000,0.000000,0.000000,0.004955,0.000000,0.00000,0.000000,0.045435,0.000000
1,0.390027,0.210884,0.269394,0.000000,0.141990,0.0725,0.002972,0.035533,0.0,0.078652,...,0.000000,0.000000,0.000000,0.000000,0.001982,0.000000,0.03876,0.257798,0.376711,0.005839
2,0.390027,0.006279,0.413348,0.000000,0.004572,0.0025,0.003715,0.005076,0.0,0.003745,...,0.000000,0.000000,0.008089,0.000000,0.000000,0.000000,0.00000,0.003819,0.000204,0.002972
3,0.390027,0.386185,0.524391,0.000000,0.186724,0.0375,0.005201,0.005076,0.0,0.074906,...,0.304527,0.051948,0.522750,0.130435,0.033697,0.162963,0.00000,0.005092,0.391638,0.007634
4,0.390027,0.092098,0.524391,0.030303,0.063248,0.0350,0.005944,0.010152,0.0,0.052434,...,0.000000,0.000000,0.000000,0.000000,0.002973,0.000000,0.00000,0.112031,0.044474,0.004249


In [208]:
testData_sinId = test_entrenamiento

In [209]:
prediction = clf.predict(testData_sinId)

In [210]:
ids = test['person'].values
dataKaggle = pd.DataFrame(ids ,columns={'person'})
dataKaggle.head(5)

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [211]:
#Aca agregamos una columna que seria se postulop
dataKaggle['label'] = prediction
dataKaggle.nsmallest(5,'label')

,person,label
2994,06ed04d6,-0.602050
260,d84ae194,-0.511979
4472,6355a425,-0.442045
5066,33385551,-0.431960
12944,a245547f,-0.405955


In [212]:
dataKaggle['label'] = dataKaggle['label'].map(lambda x: 0.0 if x < 0.0 else x)


In [213]:
dataKaggle.reset_index()

,index,person,label
0,0,4886f805,0.084486
1,1,0297fc1e,0.128648
2,2,2d681dd8,0.084074
3,3,cccea85e,0.000000
4,4,4c8a8b93,0.058505
5,5,29ebb414,0.076017
6,6,3dc1950f,0.038753
7,7,8ea4c165,0.099414
8,8,d8cfe234,0.100455
9,9,d6bc64df,0.129596


In [214]:
dataKaggle.nsmallest(5,'label')

,person,label
3,cccea85e,0.0
18,7d876393,0.0
20,5a724794,0.0
47,c76b8417,0.0
63,8af64425,0.0


In [215]:
len(dataKaggle)

19415

In [216]:
dataKaggle.to_csv("predictionRedNeuronal.csv", index = False)